In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os,json
from pathlib import Path

In [34]:
special_sep_tokens = [
    "[eoaa]",
    "[eoau]",
    "[eoda]",
    "[eodp]",
    "[eodu]",
    "[soaa]",
    "[soau]",
    "[soda]",
    "[sodp]",
    "[sodu]",
]

In [44]:
tasks = ["NLU","POL", "NLG"]
modes = ["train", "dev", "test"]

data_parentdir = Path('./data_for_clm/').absolute().resolve()
save_parentdir = Path('./data_for_seq2seq/').absolute().resolve()
for task in tasks:
    datadpath = data_parentdir.joinpath(task)
    savedpath = save_parentdir.joinpath(task)
    savedpath.mkdir(exist_ok=True)
    for mode in modes:
        mode_str = "val" if mode == "dev" else mode
        # 处理source文件
        sourcefname = f"{task}/{mode}-{task}.source"
        sourcefpath = data_parentdir.joinpath(sourcefname)
        with sourcefpath.open() as f:
            source_data = f.read().strip().splitlines()
        source_data_new = []
        for e in source_data:
            words = e.strip().split()
            if task != "POL": #delete profile
                eodp_idx = words.index("[eodp]")
                words = words[eodp_idx:]
            # words = [w for w in words if w not in special_sep_tokens]
            words = [w if w not in special_sep_tokens else " " for w in words]
            sentence = " ".join(words).strip()
            source_data_new.append(sentence)
        
        
        
        
        # 处理target文件
        targetfname = f"{task}/{mode}-{task}.target"
        targetfpath = data_parentdir.joinpath(targetfname)
        with targetfpath.open() as f:
            target_data = f.read().strip().splitlines()
        target_data_new = [e.strip().split() for e in target_data]
        #target_data_new = [[e2 for e2 in e if e2 not in special_sep_tokens] for e in target_data_new]
        target_data_new = [[e2 if e2 not in special_sep_tokens else " "for e2 in e ] for e in target_data_new]
        target_data_new = [' '.join(e) for e in target_data_new]
        target_data_new = [e.strip() for e in target_data_new]
        # 去除空行
        source_nonempty = []
        target_nonempty = []
        for sour,targ in zip(source_data_new,target_data_new):
            if sour.strip()=="" or targ.strip()=="":
                continue
            source_nonempty.append(f"{sour}\n")
            target_nonempty.append(f"{targ}\n")
        
        
        sourcesavefname = f"{mode_str}.source"
        sourcesavefpath = savedpath.joinpath(sourcesavefname)
        with sourcesavefpath.open('w') as f:
            f.writelines(source_nonempty)
        
        targetsavefname = f"{mode_str}.target"
        targetsavefpath = savedpath.joinpath(targetsavefname)
        with targetsavefpath.open('w') as f:
                f.writelines(target_nonempty)

In [42]:
["a" if x==1 else "b" for x in [1,2,3] ]

['a', 'b', 'b']

In [31]:
source_data[0]

'[sodp] [poi_type] gas stations [distance] quickest [traffic_info] avoid all heavy traffic [poi] chevron [greeting] thank [eodp] [soau] [eoau] [soaa] [eoaa] [soda] [poi_type] gas stations [eoda]'

In [32]:
target_data[0]

'[sodu] What gas stations are here? [eodu]'